# Analyzing River Thames Water Levels
Time series data is everywhere, from watching your stock portfolio to monitoring climate change, and even live-tracking as local cases of a virus become a global pandemic. In this project, you’ll work with a time series that tracks the tide levels of the Thames River. You’ll first load the data and inspect it data visually, and then perform calculations on the dataset to generate some summary statistics. You’ll end by reducing the time series to its component attributes and analyzing them. 

The original dataset is available from the British Oceanographic Data Center.

Here's a map of the locations of the tidal meters along the River Thames in London.

![](locations.png)

The provided datasets are in the `data` folder in this workspace. For this project, you will work with one of these files, `10-11_London_Bridge.txt`, which contains comma separated values for water levels in the Thames River at the London Bridge. After you've finished the project, you can use your same code to analyze data from the other files (at other spots in the UK where tidal data is collected) if you'd like. 

The TXT file contains data for three variables, described in the table below. 

| Variable Name | Description | Format |
| ------------- | ----------- | ------ |
| Date and time | Date and time of measurement to GMT. Note the tide gauge is accurate to one minute. | dd/mm/yyyy hh:mm:ss |
| Water level | High or low water level measured by tide meter. Tide gauges are accurate to 1 centimetre. | metres (Admiralty Chart Datum (CD), Ordnance Datum Newlyn (ODN or Trinity High Water (THW)) | 
| Flag | High water flag = 1, low water flag = 0 | Categorical (0 or 1) |



In [68]:
import pandas as pd
import numpy as np

In [69]:
def IQR(column): 
    """ Calculates the interquartile range (IQR) for a given DataFrame column using the quantile method """
    q25, q75 = column.quantile([0.25, 0.75])
    return q75-q25

In [70]:
df = pd.read_csv('data/10-11_London_Bridge.txt',\
                 header=0,\
                 names=['Time', 'Water Level', 'Flag', 'temp'],\
                 dtype={'Water Level': np.float64, 'Flag': 'category'},\
                 parse_dates=['Time'],\
                 index_col=['Time'],\
                 skipinitialspace=True)
df.drop(columns='temp', inplace=True)
df.head()

,Water Level,Flag
Time,,
1911-01-05 15:40:00,3.7130,1
1911-02-05 11:25:00,-2.9415,0
1911-02-05 16:05:00,3.3828,1
1911-03-05 11:50:00,-2.6367,0
1911-03-05 16:55:00,2.9256,1


In [71]:
# High Tide Flag = 1
high_df = df[df['Flag'] == '1']['Water Level']
high_tide_stats = high_df.apply([np.mean, np.median, IQR])

high_extreme_thresh = high_df.quantile(0.9)
very_high_ratios = high_df.groupby(high_df.index.year)\
                          .agg(water_level=lambda x: x[x > high_extreme_thresh].count() / x.count())\
                          .reset_index()\
                          .rename(columns={'Time': 'year'})

In [72]:
# Low Tide Flag = 0
low_df = df[df['Flag'] == '0']['Water Level']
low_tide_stats = low_df.apply([np.mean, np.median, IQR])

low_extreme_thresh = low_df.quantile(0.1)
very_low_ratios = low_df.groupby(low_df.index.year)\
                        .agg(water_level=lambda x: x[x < low_extreme_thresh].count() / x.count())\
                        .reset_index()\
                        .rename(columns={'Time': 'year'})

In [73]:
solution = {
    'high_statistics': high_tide_stats,
    'low_statistics': low_tide_stats,
    'very_high_ratio': very_high_ratios,
    'very_low_ratio': very_low_ratios
}

print(solution)

{'high_statistics': mean      3.318373
median    3.352600
IQR       0.743600
Name: Water Level, dtype: float64, 'low_statistics': mean     -2.383737
median   -2.412900
IQR       0.538200
Name: Water Level, dtype: float64, 'very_high_ratio':     year  water_level
0   1911     0.004098
1   1912     0.032316
2   1913     0.082212
3   1914     0.055313
4   1915     0.045045
..   ...          ...
80  1991     0.096317
81  1992     0.103253
82  1993     0.145923
83  1994     0.150355
84  1995     0.170213

[85 rows x 2 columns], 'very_low_ratio':     year  water_level
0   1911     0.060606
1   1912     0.066667
2   1913     0.022388
3   1914     0.039017
4   1915     0.033435
..   ...          ...
80  1991     0.150355
81  1992     0.107496
82  1993     0.112696
83  1994     0.106383
84  1995     0.107801

[85 rows x 2 columns]}
